# VDOT

This is a **mathematical exploration** of Jack Daniels and Jimmy Gilbert's $\mathrm{VDOT}$ concept.
It is based on their book _Oxygen Power (1979)_,
as well as Daniels' more recent _Daniels' Running Formula (3rd edition, 2013)_.

See also:

- Larry Simpson's [_The Daniels/Gilbert Formula_](http://www.simpsonassociatesinc.com/runningmath1.htm) web pages
- [This discussion](https://www.letsrun.com/forum/flat_read.php?thread=3704747) on letsrun.com

## Basics

The relationship between oxygen consumption $\mathrm{\dot{V}O_2}$
($\mathrm{ml \cdot kg^{-1} \cdot min^{-1}}$)
and running velocity $\mathrm{v}$
($\mathrm{m \cdot min^{-1}}$) is given by:

$$
\mathrm{\dot{V}O_2} = -4.60 + 0.182258 \mathrm{v} + 0.000104 \mathrm{v^2}
$$

The relationship between the fraction of $\mathrm{\dot{V}O_2max}$
and the duration of a distance race $\mathrm{t}$ ($\mathrm{min}$) is given by:

$$
\mathrm{F\dot{V}O_2max} = 0.8 + 0.1894393 e^{-0.012778 \mathrm{t}} + 0.2989558 e^{-0.1932605 \mathrm{t}}
$$

VDOT (pseudo $\mathrm{\dot{V}O_2max}$) can then be easily calculated as:

$$
\mathrm{VDOT} = \frac{\mathrm{\dot{V}O_2}}{\mathrm{F\dot{V}O_2max}}
$$

---

Below is the $\mathrm{VDOT}$ calculation of a runner that ran a **5k race in 24 minutes and 34 seconds**.

In [1]:
import math
from datetime import timedelta

# 5k race with a time of 24:34
d = 5000
t = timedelta(minutes=24, seconds=34).total_seconds() / 60
v = d / t

vo2 = -4.6 + 0.182258 * v + 0.000104 * v**2
pct = 0.8 + 0.1894393 * math.exp(-0.012778 * t) + 0.2989558 * math.exp(-0.1932605 * t)

vdot = vo2 / pct
vdot

39.11039839652726

Looks about right from inspecting Daniels' $\mathrm{VDOT}$ tables:

| VDOT | 1,500 | Mile | 3,000 | 2 mile | 5,000 | 10K   | 15K   | Half marathon | Marathon |
| ---- | ----- | ---- | ----- | ------ | ----- | ----- | ----- | ------------- | -------- |
| ...  |
| 38   | 6:54  | 7:27 | 14:41 | 15:49  | **25:12** | 52:17 | 80:33 | 1:55:55       | 3:59:35  |
| 39   | 6:44  | 7:17 | 14:21 | 15:29  | **24:39** | 51:09 | 78:47 | 1:53:24       | 3:54:34  |
| 40   | 6:35  | 7:07 | 14:03 | 15:08  | **24:08** | 50:03 | 77:06 | 1:50:59       | 3:49:45  |
| ...  |

## Estimating other race performances

The formulas presented above can be used to estimate various race times
once a $\mathrm{VDOT}$ value as been calculated.

This can be done by finding the root of $f(d, t)$ (while setting $d$ to the desired race distance):

$$
f(d, t) = \frac{-4.6 + 0.182258 d t^{-1} + 0.000104 d^2 t^{-2}}{0.8 + 0.1894393 e^{-0.012778 \mathrm{t}} + 0.2989558 e^{-0.1932605 \mathrm{t}}} - \mathrm{VDOT}
$$

This is a root finding problem of a non-linear equation that can be solved with iterative methods.

In [2]:
distances = {
    "Marathon": 42195,
    "Half marathon": 21097.5,
    "15K": 15000,
    "10K": 10000,
    "5K": 5000,
}


def format_duration(td):
    s = int(td.total_seconds())
    hh, mm, ss = s // 3600, s // 60 % 60, s % 60

    if hh > 0:
        return f"{hh:02}:{mm:02}:{ss:02}"

    return f"{mm:02}:{ss:02}"

Using the bisection method:

In [3]:
from scipy import optimize


def f(x, vdot, d):
    return (-4.6 + 0.182258 * d * x ** (-1) + 0.000104 * d**2 * x ** (-2)) / (
        0.8 + 0.1894393 * math.exp(-0.012778 * x) + 0.2989558 * math.exp(-0.1932605 * x)
    ) - vdot


for name, d in distances.items():
    root = optimize.bisect(f, 1, 600, args=(vdot, d))
    td = timedelta(minutes=root)

    td_s = format_duration(td)
    print(f"{name:14}: {td_s}")

Marathon      : 03:53:54
Half marathon : 01:53:02
15K           : 01:18:32
10K           : 50:58
5K            : 24:34


These results seem to deviate a couple of seconds from
["official" calculators](https://vdoto2.com/Calculator)
or Daniels' own tables;
I believe this is to be expected as **the calculator does not match
exactly with the tables** from _Daniels' Running Formula (3rd edition, 2013)_.

I assume the equations' parameters have been refined since 1979 and have
not been made public. Either that or I fucked up somewhere. ¯\\_(ツ)_/¯

## Defining training paces

The calculation of training paces is not explained or talked about in _Oxygen Power_.
However, in "Chapter 4: Training Runs and Itensities" of _Daniels' Running Formula_
he sets the following intensities (based on percentages of a runner's $\mathrm{\dot{V}O_2max}$):

- Easy: 59-74%
- Marathon: 75-84%
- Threshold: 83-88%
- Interval: 95-100%
- Repetition: 105-120%

Using the above defined relationship between speed and oxygen consumption,
the pace ranges can be calculated:

$$
\mathrm{v} = \frac{-0.182258 + \sqrt{0.033218 - 0.000416 (-4.6 -\mathrm{\dot{V}O_2})}}{0.000208}
$$

In [4]:
paces = {
    "Easy": (0.59, 0.74),
    "Marathon": (0.75, 0.84),
    "Threshold": (0.83, 0.88),
    "Interval": (0.95, 1),
    "Repetition": (1.05, 1.2),
}


def f(vdot, pct):
    return (
        -0.182258 + math.sqrt(0.033218 - 0.000416 * (-4.6 - (vdot * pct)))
    ) / 0.000208

for name, pcts in paces.items():
    p = [f(vdot, pct) for pct in pcts]
    p = [1000 / pace for pace in p]  # min/km instead of m/min
    p = [format_duration(timedelta(minutes=pace)) for pace in p]  # "m:ss" format

    slower, faster = p
    print(f"{name:11}: {faster} ~ {slower}")

Easy       : 05:57 ~ 07:06
Marathon   : 05:22 ~ 05:53
Threshold  : 05:11 ~ 05:26
Interval   : 04:40 ~ 04:52
Repetition : 04:02 ~ 04:29


These don't match exactly to the tables or calculator (which only use a range for easy pace),
but those values are contained within the calculated ranges. I guess that's something.

When making a tool or suggesting a specific pace,
a decent estimate could be obtained by using the middle value of the
$\mathrm{\dot{V}O_2}$ range (e.g. 66.5% for easy pace).